## %pip install numpy pyvista simpleitk monai nibabel fire mlflow

# %% Imports
import os
import json
import torch
import numpy as np
import SimpleITK as sitk
import nibabel as nib
import matplotlib.pyplot as plt

from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config

print_config()

# %% [markdown]
# # Create Updated JSON Datalist

# Define dataset splits (update file paths)
sim_datalist = {
    "testing": [
        {"image": "imagesTr/Normal_5.nii.gz", "label": "labelsTr/Normal_5.nii.gz"},
        {"image": "imagesTr/Diseased_5.nii.gz", "label": "labelsTr/Diseased_5.nii.gz"},
        {"image": "imagesTr/Normal_10.nii.gz", "label": "labelsTr/Normal_10.nii.gz"},
        {"image": "imagesTr/Diseased_10.nii.gz", "label": "labelsTr/Diseased_10.nii.gz"},
        {"image": "imagesTr/Normal_15.nii.gz", "label": "labelsTr/Normal_15.nii.gz"},
        {"image": "imagesTr/Diseased_15.nii.gz", "label": "labelsTr/Diseased_15.nii.gz"},
        {"image": "imagesTr/Normal_20.nii.gz", "label": "labelsTr/Normal_20.nii.gz"},
        {"image": "imagesTr/Diseased_20.nii.gz", "label": "labelsTr/Diseased_20.nii.gz"}
    ],
    "training": [
        {"fold": 0, "image": "imagesTr/Normal_1.nii.gz", "label": "labelsTr/Normal_1.nii.gz"},
        {"fold": 0, "image": "imagesTr/Normal_2.nii.gz", "label": "labelsTr/Normal_2.nii.gz"},
        {"fold": 0, "image": "imagesTr/Normal_3.nii.gz", "label": "labelsTr/Normal_3.nii.gz"},
        {"fold": 0, "image": "imagesTr/Normal_4.nii.gz", "label": "labelsTr/Normal_4.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_6.nii.gz", "label": "labelsTr/Normal_6.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_7.nii.gz", "label": "labelsTr/Normal_7.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_8.nii.gz", "label": "labelsTr/Normal_8.nii.gz"},
        {"fold": 1, "image": "imagesTr/Normal_9.nii.gz", "label": "labelsTr/Normal_9.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_11.nii.gz", "label": "labelsTr/Normal_11.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_12.nii.gz", "label": "labelsTr/Normal_12.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_13.nii.gz", "label": "labelsTr/Normal_13.nii.gz"},
        {"fold": 2, "image": "imagesTr/Normal_14.nii.gz", "label": "labelsTr/Normal_14.nii.gz"},    
        {"fold": 3, "image": "imagesTr/Normal_16.nii.gz", "label": "labelsTr/Normal_16.nii.gz"},
        {"fold": 3, "image": "imagesTr/Normal_17.nii.gz", "label": "labelsTr/Normal_17.nii.gz"},
        {"fold": 3, "image": "imagesTr/Normal_18.nii.gz", "label": "labelsTr/Normal_18.nii.gz"},
        {"fold": 3, "image": "imagesTr/Normal_19.nii.gz", "label": "labelsTr/Normal_19.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_1.nii.gz", "label": "labelsTr/Diseased_1.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_2.nii.gz", "label": "labelsTr/Diseased_2.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_3.nii.gz", "label": "labelsTr/Diseased_3.nii.gz"},
        {"fold": 0, "image": "imagesTr/Diseased_4.nii.gz", "label": "labelsTr/Diseased_4.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_6.nii.gz", "label": "labelsTr/Diseased_6.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_7.nii.gz", "label": "labelsTr/Diseased_7.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_8.nii.gz", "label": "labelsTr/Diseased_8.nii.gz"},
        {"fold": 1, "image": "imagesTr/Diseased_9.nii.gz", "label": "labelsTr/Diseased_9.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_11.nii.gz", "label": "labelsTr/Diseased_11.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_12.nii.gz", "label": "labelsTr/Diseased_12.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_13.nii.gz", "label": "labelsTr/Diseased_13.nii.gz"},
        {"fold": 2, "image": "imagesTr/Diseased_14.nii.gz", "label": "labelsTr/Diseased_14.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_16.nii.gz", "label": "labelsTr/Diseased_16.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_17.nii.gz", "label": "labelsTr/Diseased_17.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_18.nii.gz", "label": "labelsTr/Diseased_18.nii.gz"},
        {"fold": 3, "image": "imagesTr/Diseased_19.nii.gz", "label": "labelsTr/Diseased_19.nii.gz"}
    ]
}

# Save datalist JSON
datalist_file = os.path.join("./2ndauto3dseg_dir", "sim_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(sim_datalist, f)

print(f"✅ Datalist saved at: {datalist_file}")

work_dir = "./2ndauto3dseg_dir"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = os.path.join(work_dir, "sim_dataroot")
#dataroot_dir = work_dir
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "sim_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(sim_datalist, f)


In [2]:
# Define AutoRunner
input={
        "modality": "CT",
        "datalist": datalist_file,
        "dataroot": dataroot_dir,
    }

output_data_dir = "./2ndauto3dseg_dir/sim_dataroot"

runner = AutoRunner(
    input=input,
    work_dir="./2ndauto3dseg_dir",
    datalist_filename=datalist_file,  # Updated JSON with relative paths
    dataroot=output_data_dir,  # Directory containing converted .nii.gz files
    #analyze_params={"image_key":"image","label_key":"label"},
    algos=['dints']
)

# %% [markdown]
# # Configure Training Parameters

max_epochs = 100
train_param = {
    "num_epochs_per_validation": 5,
    "num_images_per_batch": 1, #If you want to run with more images per batch, upgrade of the UT shared memory is needed. More than 1 image per batch currently results in a cuda out of memory error.
    "num_epochs": max_epochs,
    "num_warmup_epochs": 2,
}

runner.set_training_params(train_param)
runner.set_num_fold(num_fold=4)

# %% [markdown]
# # Run AutoRunner

print(torch.cuda.device_count())

import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2, 3, 4"
print(torch.cuda.device_count())
print(f"Selected algorithms: {runner.algos}")

runner.run()

2025-04-01 16:29:39,664 - INFO - AutoRunner using work directory /home/jovyan/2ndauto3dseg_dir
2025-04-01 16:29:39,858 - INFO - Found num_fold 4 based on the input datalist /home/jovyan/2ndauto3dseg_dir/sim_datalist.json.
2025-04-01 16:29:39,860 - INFO - Setting num_fold 4 based on the input datalist /home/jovyan/2ndauto3dseg_dir/sim_datalist.json.
2025-04-01 16:29:39,866 - INFO - Using user defined command running prefix , will override other settings
8
8
Selected algorithms: ['dints']
2025-04-01 16:29:39,869 - INFO - Skipping data analysis...
2025-04-01 16:29:39,870 - INFO - Skipping algorithm generation...
2025-04-01 16:29:39,924 - INFO - Skipping already trained algos ['dints_0', 'dints_1'].Set option train=True to always retrain all algos.
2025-04-01 16:29:39,945 - INFO - The keys num_warmup_epochs cannot be found in the /home/jovyan/2ndauto3dseg_dir/dints_2/configs/hyper_parameters.yaml for training. Skipped overriding key num_warmup_epochs.
2025-04-01 16:29:39,950 - INFO - ['tor

W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] 
W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] *****************************************
W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0401 16:29:41.857000 140538417358656 torch/distributed/run.py:757] *****************************************
/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/jovyan/.local/lib/python3.10/site-packages/matplotli

dints_2 - validation at original spacing/resolution


2025-04-01 17:42:32,070 - WARNING - dints_2 - training: finished


2025-04-01 17:42:51,086 - INFO - The keys num_warmup_epochs cannot be found in the /home/jovyan/2ndauto3dseg_dir/dints_3/configs/hyper_parameters.yaml for training. Skipped overriding key num_warmup_epochs.
2025-04-01 17:42:51,103 - INFO - ['torchrun', '--nnodes', '1', '--nproc_per_node', '8', '/home/jovyan/2ndauto3dseg_dir/dints_3/scripts/train.py', 'run', "--config_file='/home/jovyan/2ndauto3dseg_dir/dints_3/configs/hyper_parameters.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/hyper_parameters_search.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/network.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/network_search.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/transforms_infer.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/transforms_train.yaml,/home/jovyan/2ndauto3dseg_dir/dints_3/configs/transforms_validate.yaml'", '--training#num_epochs_per_validation=5', '--training#num_images_per_batch=1', '--training#num_epochs=100']


...

2025-04-01 18:51:13,833 - INFO - The output_dir is not specified. /home/jovyan/2ndauto3dseg_dir/ensemble_output will be used to save ensemble predictions.
2025-04-01 18:51:13,956 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-04-01 18:51:13,956 - INFO - dints_0
2025-04-01 18:51:13,956 - INFO - dints_1
2025-04-01 18:51:13,956 - INFO - dints_2
2025-04-01 18:51:13,957 - INFO - dints_3
2025-04-01 18:51:13,957 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/jovyan/2ndauto3dseg_dir/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/1 [00:00<?, ?it/s]monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.cr

2025-04-01 18:51:25.697472 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.697526 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.698279 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.699732 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.700198 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.705454 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.707659 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:51:25.708887 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:53:58.168757 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:54:12.094696 - Length of input patch is recommended to be a multiple of 32.
2025-04-01 18:54:44.607075 - Length of input patch is recommended to be a multiple of 32.
2025-04-01

Ensembling (rank 0)...: 100%|██████████| 1/1 [14:16<00:00, 856.86s/it]


2025-04-01 19:07:10,774 - INFO - Auto3Dseg pipeline is completed successfully.


In [1]:
#For training DiNTS on cropped data

%pip install numpy pyvista simpleitk monai nibabel fire mlflow

# %% Imports
import os
import json
import torch
import numpy as np
import SimpleITK as sitk
import nibabel as nib
import matplotlib.pyplot as plt

from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config

print_config()

# %% [markdown]
# # Create Updated JSON Datalist

# Define dataset splits (update file paths)
crop_datalist = {
    #Load in the non-cropped versions of the scans for testing, the cropped versions for training to see the effect on the performance metrics
     "testing": [
        {"image": "imagesTr/Normal_5.nii.gz", "label": "labelsTr/Normal_5.nii.gz"},
        {"image": "imagesTr/Diseased_5.nii.gz", "label": "labelsTr/Diseased_5.nii.gz"},
        {"image": "imagesTr/Normal_10.nii.gz", "label": "labelsTr/Normal_10.nii.gz"},
        {"image": "imagesTr/Diseased_10.nii.gz", "label": "labelsTr/Diseased_10.nii.gz"},
        {"image": "imagesTr/Normal_15.nii.gz", "label": "labelsTr/Normal_15.nii.gz"},
        {"image": "imagesTr/Diseased_15.nii.gz", "label": "labelsTr/Diseased_15.nii.gz"},
        {"image": "imagesTr/Normal_20.nii.gz", "label": "labelsTr/Normal_20.nii.gz"},
        {"image": "imagesTr/Diseased_20.nii.gz", "label": "labelsTr/Diseased_20.nii.gz"}
    ],
    "training": [
        {"fold": 0, "image": "imagesTr/ROI_CTCA1.nii.gz", "label": "labelsTr/ROI_annotation1.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_CTCA2.nii.gz", "label": "labelsTr/ROI_annotation2.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_CTCA3.nii.gz", "label": "labelsTr/ROI_annotation3.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_CTCA4.nii.gz", "label": "labelsTr/ROI_annotation4.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_CTCA6.nii.gz", "label": "labelsTr/ROI_annotation6.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_CTCA7.nii.gz", "label": "labelsTr/ROI_annotation7.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_CTCA8.nii.gz", "label": "labelsTr/ROI_annotation8.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_CTCA9.nii.gz", "label": "labelsTr/ROI_annotation9.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_CTCA11.nii.gz", "label": "labelsTr/ROI_annotation11.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_CTCA12.nii.gz", "label": "labelsTr/ROI_annotation12.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_CTCA13.nii.gz", "label": "labelsTr/ROI_annotation13.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_CTCA14.nii.gz", "label": "labelsTr/ROI_annotation14.nii.gz"},    
        {"fold": 3, "image": "imagesTr/ROI_CTCA16.nii.gz", "label": "labelsTr/ROI_annotation16.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_CTCA17.nii.gz", "label": "labelsTr/ROI_annotation17.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_CTCA18.nii.gz", "label": "labelsTr/ROI_annotation18.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_CTCA19.nii.gz", "label": "labelsTr/ROI_annotation19.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_volume1.nii.gz", "label": "labelsTr/ROI_annotationD1.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_volume2.nii.gz", "label": "labelsTr/ROI_annotationD2.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_volume3.nii.gz", "label": "labelsTr/ROI_annotationD3.nii.gz"},
        {"fold": 0, "image": "imagesTr/ROI_volume4.nii.gz", "label": "labelsTr/ROI_annotationD4.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_volume6.nii.gz", "label": "labelsTr/ROI_annotationD6.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_volume7.nii.gz", "label": "labelsTr/ROI_annotationD7.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_volume8.nii.gz", "label": "labelsTr/ROI_annotationD8.nii.gz"},
        {"fold": 1, "image": "imagesTr/ROI_volume9.nii.gz", "label": "labelsTr/ROI_annotationD9.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_volume11.nii.gz", "label": "labelsTr/ROI_annotationD11.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_volume12.nii.gz", "label": "labelsTr/ROI_annotationD12.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_volume13.nii.gz", "label": "labelsTr/ROI_annotationD13.nii.gz"},
        {"fold": 2, "image": "imagesTr/ROI_volume14.nii.gz", "label": "labelsTr/ROI_annotationD14.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_volume16.nii.gz", "label": "labelsTr/ROI_annotationD16.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_volume17.nii.gz", "label": "labelsTr/ROI_annotationD17.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_volume18.nii.gz", "label": "labelsTr/ROI_annotationD18.nii.gz"},
        {"fold": 3, "image": "imagesTr/ROI_volume19.nii.gz", "label": "labelsTr/ROI_annotationD19.nii.gz"}
    ]
}

# Save datalist JSON
datalist_file = os.path.join("./3rdauto3dseg_dir", "crop_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(crop_datalist, f)

print(f"✅ Datalist saved at: {datalist_file}")

work_dir = "./3rdauto3dseg_dir"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = os.path.join(work_dir, "crop_dataroot")
#dataroot_dir = work_dir
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-04-13 21:51:35.215290: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 21:51:35.266027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-13 21:51:35.266054: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT fa

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.3.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.3
Nibabel version: 5.3.2
scikit-image version: 0.25.1
scipy version: 1.15.2
Pillow version: 11.1.0
Tensorboard version: 2.14.1
gdown version: 5.2.0
TorchVision version: 0.18.1+cu118
tqdm version: 4.67.1
lmdb version: 1.6.2
psutil version: 7.0.0
pandas version: 2.2.3
einops version: 0.8.1
transformers version: 4.40.2
mlflow version: 2.21.0
pynrrd version: 1.1.3
clearml version: 1.18.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

✅ Datalist saved at: ./3rdauto3dseg_dir/crop_datalist.json


In [6]:
# Define AutoRunner
input={
        "modality": "CT",
        "datalist": datalist_file,
        "dataroot": dataroot_dir,
    }

output_data_dir = "./3rdauto3dseg_dir/crop_dataroot"

runner = AutoRunner(
    input=input,
    work_dir="./3rdauto3dseg_dir",
    datalist_filename=datalist_file,  # Updated JSON with relative paths
    dataroot=output_data_dir,  # Directory containing converted .nii.gz files
    #analyze_params={"image_key":"image","label_key":"label"},
    algos=['dints']
)

# %% [markdown]
# # Configure Training Parameters

max_epochs = 100
train_param = {
    "num_epochs_per_validation": 5,
    "num_images_per_batch": 1,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 2,
}

runner.set_training_params(train_param)
runner.set_num_fold(num_fold=4)

# %% [markdown]
# # Run AutoRunner

print(torch.cuda.device_count())

import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2, 3, 4"
print(torch.cuda.device_count())
print(f"Selected algorithms: {runner.algos}")

runner.run()

2025-04-13 22:05:38,120 - INFO - AutoRunner using work directory /home/jovyan/3rdauto3dseg_dir
2025-04-13 22:05:38,208 - INFO - Found num_fold 4 based on the input datalist /home/jovyan/3rdauto3dseg_dir/crop_datalist.json.
2025-04-13 22:05:38,209 - INFO - Setting num_fold 4 based on the input datalist /home/jovyan/3rdauto3dseg_dir/crop_datalist.json.
2025-04-13 22:05:38,213 - INFO - Using user defined command running prefix , will override other settings
8
8
Selected algorithms: ['dints']
2025-04-13 22:05:38,215 - INFO - Skipping data analysis...
2025-04-13 22:05:38,215 - INFO - Skipping algorithm generation...
2025-04-13 22:05:38,313 - INFO - The keys num_warmup_epochs cannot be found in the /home/jovyan/3rdauto3dseg_dir/dints_0/configs/hyper_parameters.yaml for training. Skipped overriding key num_warmup_epochs.
2025-04-13 22:05:38,316 - INFO - ['torchrun', '--nnodes', '1', '--nproc_per_node', '4', '/home/jovyan/3rdauto3dseg_dir/dints_0/scripts/train.py', 'run', "--config_file='/home

W0413 22:05:40.272000 140368880146240 torch/distributed/run.py:757] 
W0413 22:05:40.272000 140368880146240 torch/distributed/run.py:757] *****************************************
W0413 22:05:40.272000 140368880146240 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0413 22:05:40.272000 140368880146240 torch/distributed/run.py:757] *****************************************
/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/jovyan/.local/lib/python3.10/site-packages/matplotli

dints_0 - validation at original spacing/resolution


2025-04-13 23:46:01,180 - WARNING - dints_0 - training: finished


2025-04-13 23:46:18,488 - INFO - The keys num_warmup_epochs cannot be found in the /home/jovyan/3rdauto3dseg_dir/dints_1/configs/hyper_parameters.yaml for training. Skipped overriding key num_warmup_epochs.
2025-04-13 23:46:18,516 - INFO - ['torchrun', '--nnodes', '1', '--nproc_per_node', '4', '/home/jovyan/3rdauto3dseg_dir/dints_1/scripts/train.py', 'run', "--config_file='/home/jovyan/3rdauto3dseg_dir/dints_1/configs/hyper_parameters.yaml,/home/jovyan/3rdauto3dseg_dir/dints_1/configs/hyper_parameters_search.yaml,/home/jovyan/3rdauto3dseg_dir/dints_1/configs/network.yaml,/home/jovyan/3rdauto3dseg_dir/dints_1/configs/network_search.yaml,/home/jovyan/3rdauto3dseg_dir/dints_1/configs/transforms_infer.yaml,/home/jovyan/3rdauto3dseg_dir/dints_1/configs/transforms_train.yaml,/home/jovyan/3rdauto3dseg_dir/dints_1/configs/transforms_validate.yaml'", '--training#num_epochs_per_validation=5', '--training#num_images_per_batch=1', '--training#num_epochs=100']


...

2025-04-14 04:18:59.851888 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:18:59.853562 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:18:59.856463 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:18:59.863084 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:21:33.998908 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:22:08.677748 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:22:08.888822 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:22:08.893185 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:24:40.877283 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:25:17.163810 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:25:17.622765 - Length of input patch is recommended to be a multiple of 32.
2025-04-14

Ensembling (rank 0)...:  50%|█████     | 1/2 [11:00<11:00, 660.55s/it]

2025-04-14 04:31:06.288522 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:31:41.614609 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:31:41.906422 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:31:41.909865 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:34:26.822526 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:35:00.858613 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:35:13.579472 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:35:13.582200 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:37:57.231117 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:38:31.642556 - Length of input patch is recommended to be a multiple of 32.
2025-04-14 04:38:45.898636 - Length of input patch is recommended to be a multiple of 32.
2025-04-14

Ensembling (rank 0)...: 100%|██████████| 2/2 [25:42<00:00, 771.45s/it]


2025-04-14 04:46:08,117 - INFO - Auto3Dseg pipeline is completed successfully.


In [1]:
#For training DiNTS on augmented data

%pip install numpy pyvista simpleitk monai nibabel fire mlflow

# %% Imports
import os
import json
import torch
import numpy as np
import SimpleITK as sitk
import nibabel as nib
import matplotlib.pyplot as plt

from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config

print_config()

# %% [markdown]
# # Create Updated JSON Datalist

# Define dataset splits (update file paths)
aug_datalist = {
    #Load in the non-cropped versions of the scans for testing, the cropped augmented versions for training to see the effect on the performance metrics
     "testing": [
        {"image": "imagesTr/Normal_5.nii.gz", "label": "labelsTr/Normal_5.nii.gz"},
        {"image": "imagesTr/Diseased_5.nii.gz", "label": "labelsTr/Diseased_5.nii.gz"},
        {"image": "imagesTr/Normal_10.nii.gz", "label": "labelsTr/Normal_10.nii.gz"},
        {"image": "imagesTr/Diseased_10.nii.gz", "label": "labelsTr/Diseased_10.nii.gz"},
        {"image": "imagesTr/Normal_15.nii.gz", "label": "labelsTr/Normal_15.nii.gz"},
        {"image": "imagesTr/Diseased_15.nii.gz", "label": "labelsTr/Diseased_15.nii.gz"},
        {"image": "imagesTr/Normal_20.nii.gz", "label": "labelsTr/Normal_20.nii.gz"},
        {"image": "imagesTr/Diseased_20.nii.gz", "label": "labelsTr/Diseased_20.nii.gz"}
    ],
    "training": [
    {"fold": 0, "image": "imagesTr/aug_image_001.nii.gz", "label": "labelsTr/aug_label_001.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_002.nii.gz", "label": "labelsTr/aug_label_002.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_003.nii.gz", "label": "labelsTr/aug_label_003.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_004.nii.gz", "label": "labelsTr/aug_label_004.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_006.nii.gz", "label": "labelsTr/aug_label_006.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_007.nii.gz", "label": "labelsTr/aug_label_007.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_008.nii.gz", "label": "labelsTr/aug_label_008.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_009.nii.gz", "label": "labelsTr/aug_label_009.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_011.nii.gz", "label": "labelsTr/aug_label_011.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_012.nii.gz", "label": "labelsTr/aug_label_012.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_013.nii.gz", "label": "labelsTr/aug_label_013.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_014.nii.gz", "label": "labelsTr/aug_label_014.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_016.nii.gz", "label": "labelsTr/aug_label_016.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_017.nii.gz", "label": "labelsTr/aug_label_017.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_018.nii.gz", "label": "labelsTr/aug_label_018.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_019.nii.gz", "label": "labelsTr/aug_label_019.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_021.nii.gz", "label": "labelsTr/aug_label_021.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_022.nii.gz", "label": "labelsTr/aug_label_022.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_023.nii.gz", "label": "labelsTr/aug_label_023.nii.gz"},
    {"fold": 0, "image": "imagesTr/aug_image_024.nii.gz", "label": "labelsTr/aug_label_024.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_026.nii.gz", "label": "labelsTr/aug_label_026.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_027.nii.gz", "label": "labelsTr/aug_label_027.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_028.nii.gz", "label": "labelsTr/aug_label_028.nii.gz"},
    {"fold": 1, "image": "imagesTr/aug_image_029.nii.gz", "label": "labelsTr/aug_label_029.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_031.nii.gz", "label": "labelsTr/aug_label_031.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_032.nii.gz", "label": "labelsTr/aug_label_032.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_033.nii.gz", "label": "labelsTr/aug_label_033.nii.gz"},
    {"fold": 2, "image": "imagesTr/aug_image_034.nii.gz", "label": "labelsTr/aug_label_034.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_036.nii.gz", "label": "labelsTr/aug_label_036.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_037.nii.gz", "label": "labelsTr/aug_label_037.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_038.nii.gz", "label": "labelsTr/aug_label_038.nii.gz"},
    {"fold": 3, "image": "imagesTr/aug_image_039.nii.gz", "label": "labelsTr/aug_label_039.nii.gz"}
]
}

# Save datalist JSON
datalist_file = os.path.join("./4thauto3dseg_dir", "aug_datalist.json")
with open(datalist_file, "w") as f:
    json.dump(aug_datalist, f)

print(f"✅ Datalist saved at: {datalist_file}")

work_dir = "./4thauto3dseg_dir"
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

dataroot_dir = os.path.join(work_dir, "aug_dataroot")
#dataroot_dir = work_dir
if not os.path.isdir(dataroot_dir):
    os.makedirs(dataroot_dir)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-04-16 10:35:39.503981: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 10:35:39.540920: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 10:35:39.540944: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT fa

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.3.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.3
Nibabel version: 5.3.2
scikit-image version: 0.25.1
scipy version: 1.15.2
Pillow version: 11.1.0
Tensorboard version: 2.14.1
gdown version: 5.2.0
TorchVision version: 0.18.1+cu118
tqdm version: 4.67.1
lmdb version: 1.6.2
psutil version: 7.0.0
pandas version: 2.2.3
einops version: 0.8.1
transformers version: 4.40.2
mlflow version: 2.21.0
pynrrd version: 1.1.3
clearml version: 1.18.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

✅ Datalist saved at: ./4thauto3dseg_dir/aug_datalist.json


In [5]:
# Define AutoRunner
input={
        "modality": "CT",
        "datalist": datalist_file,
        "dataroot": dataroot_dir,
    }

output_data_dir = "./4thauto3dseg_dir/aug_dataroot"

runner = AutoRunner(
    input=input,
    work_dir="./4thauto3dseg_dir",
    datalist_filename=datalist_file,  # Updated JSON with relative paths
    dataroot=output_data_dir,  # Directory containing converted .nii.gz files
    #analyze_params={"image_key":"image","label_key":"label"},
    algos=['dints']
)

# %% [markdown]
# # Configure Training Parameters

max_epochs = 100
train_param = {
    "num_epochs_per_validation": 5,
    "num_images_per_batch": 1,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 2,
}

runner.set_training_params(train_param)
runner.set_num_fold(num_fold=4)

# %% [markdown]
# # Run AutoRunner

print(torch.cuda.device_count())

import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2, 3, 4"
print(torch.cuda.device_count())
print(f"Selected algorithms: {runner.algos}")

runner.run()

2025-04-16 15:27:18,611 - INFO - AutoRunner using work directory /home/jovyan/4thauto3dseg_dir
2025-04-16 15:27:18,654 - INFO - Found num_fold 4 based on the input datalist /home/jovyan/4thauto3dseg_dir/aug_datalist.json.
2025-04-16 15:27:18,656 - INFO - Setting num_fold 4 based on the input datalist /home/jovyan/4thauto3dseg_dir/aug_datalist.json.
2025-04-16 15:27:18,670 - INFO - Using user defined command running prefix , will override other settings
8
8
Selected algorithms: ['dints']
2025-04-16 15:27:18,673 - INFO - Skipping data analysis...
2025-04-16 15:27:18,675 - INFO - Skipping algorithm generation...
2025-04-16 15:27:18,677 - INFO - Skipping algorithm training...
2025-04-16 15:27:18,679 - INFO - Ensembling using 4 GPU!
2025-04-16 15:27:18,681 - INFO - ['torchrun', '--nnodes', '1', '--nproc_per_node', '4', '-m', 'monai.apps.auto3dseg', 'EnsembleRunner', 'ensemble', '--data_src_cfg_name', '/home/jovyan/4thauto3dseg_dir/input.yaml', '--work_dir', '/home/jovyan/4thauto3dseg_dir', 

W0416 15:27:20.139000 140388296800064 torch/distributed/run.py:757] 
W0416 15:27:20.139000 140388296800064 torch/distributed/run.py:757] *****************************************
W0416 15:27:20.139000 140388296800064 torch/distributed/run.py:757] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0416 15:27:20.139000 140388296800064 torch/distributed/run.py:757] *****************************************
/home/jovyan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/jovyan/.local/lib/python3.10/site-packages/matplotli

2025-04-16 15:27:40,565 - INFO - The output_dir is not specified. /home/jovyan/4thauto3dseg_dir/ensemble_output will be used to save ensemble predictions.
2025-04-16 15:27:40,683 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-04-16 15:27:40,683 - INFO - dints_0
2025-04-16 15:27:40,683 - INFO - dints_1
2025-04-16 15:27:40,683 - INFO - dints_2
2025-04-16 15:27:40,683 - INFO - dints_3
2025-04-16 15:27:40,683 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/jovyan/4thauto3dseg_dir/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/2 [00:00<?, ?it/s]monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


2025-04-16 15:27:43.922993 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:27:43.940147 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:27:43.941427 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:27:43.951975 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:29:54.368752 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:29:59.148836 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:30:19.139459 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:30:19.156081 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:32:26.726774 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:32:31.806403 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:32:53.587101 - Length of input patch is recommended to be a multiple of 32.
2025-04-16

Ensembling (rank 0)...:  50%|█████     | 1/2 [09:08<09:08, 548.12s/it]

2025-04-16 15:37:42.635054 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:37:47.443856 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:38:08.653526 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:38:08.656607 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:40:28.239649 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:40:40.131339 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:40:53.461292 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:40:53.463741 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:43:13.854413 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:43:24.434349 - Length of input patch is recommended to be a multiple of 32.
2025-04-16 15:43:39.526018 - Length of input patch is recommended to be a multiple of 32.
2025-04-16

Ensembling (rank 0)...: 100%|██████████| 2/2 [20:36<00:00, 618.19s/it]


2025-04-16 15:49:26,740 - INFO - Auto3Dseg pipeline is completed successfully.
